In [46]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import optimize
from mpl_toolkits import mplot3d
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import pandas as pd
from matplotlib import rc, animation
import statsmodels.api as sm
from scipy import stats
import est_dir
import time
import tqdm

# General example for RSM

In [47]:
m = 100
n = 16
lambda_2 = 8

In [48]:
noise_list = np.genfromtxt('noise_list_n=%s_m=%s_lambda_max=%s.csv' %
                                (n, m, lambda_2), delimiter=',')
noise_list

array([12.14062262, 38.3920197 , 54.29451494, 66.49692872])

In [49]:
noise_sd = noise_list[0]
noise_sd

12.140622621224393

In [50]:
max_func_evals_t_list = [30000, 40000, 50000, 60000,70000,80000]
num = 100
domain = (0, 5)
sp_norms = np.zeros((len(max_func_evals_t_list), num))
fp_norms = np.zeros((2, len(max_func_evals_t_list), num))
sp_func_vals_noise = np.zeros((2, len(max_func_evals_t_list), num))
fp_func_vals_noise = np.zeros((2, len(max_func_evals_t_list), num))
sp_func_vals = np.zeros((len(max_func_evals_t_list), num))
fp_func_vals = np.zeros((2, len(max_func_evals_t_list), num))
time_taken = np.zeros((2, len(max_func_evals_t_list), num))
func_evals_step = np.zeros((2, len(max_func_evals_t_list), num)) 
func_evals_dir = np.zeros((2, len(max_func_evals_t_list), num))
no_pos_missed = np.zeros((len(max_func_evals_t_list), num))
index = 0
for max_func_evals_t in max_func_evals_t_list:
    for j in tqdm.tqdm(range(num)):
        seed = j * 50
        np.random.seed(seed)
        f = est_dir.sphere_f_noise
        minimizer = np.random.uniform(*domain, (m, ))
        centre_point = np.random.uniform(*domain, (m, ))
        matrix = est_dir.sphere_func_params(1, lambda_2, m)
        sp_norms[index, j] = np.linalg.norm(minimizer - centre_point)
        sp_func_vals[index, j] = est_dir.sphere_f(centre_point, minimizer, matrix)
        func_args = (minimizer, matrix, 0, noise_sd)
                        
        np.random.seed(seed + 1)
        (upd_point_LS,
         sp_func_vals_noise[0, index, j],
         fp_func_vals_noise[0, index, j],
         full_time_LS,
         func_evals_step[0, index, j],
         func_evals_dir[0, index, j],
         no_its_LS) = est_dir.calc_its_until_sc(centre_point, f, func_args, n, m,
                                                              option='LS',
                                                              max_func_evals=max_func_evals_t)

        fp_norms[0, index, j] = np.linalg.norm(minimizer - upd_point_LS)
        fp_func_vals[0, index, j] =  est_dir.sphere_f(upd_point_LS, minimizer, matrix)                                                 
        np.random.seed(seed + 1)
        (upd_point_XY, 
         sp_func_vals_noise[1, index, j],
         fp_func_vals_noise[1, index, j],
         full_time_XY,
         func_evals_step[1, index, j],
         func_evals_dir[1, index, j],
         its_XY) = est_dir.calc_its_until_sc(centre_point, f, func_args, n, m, 
                                             option='XY',
                                             max_func_evals=max_func_evals_t)
        fp_norms[1, index, j] = np.linalg.norm(minimizer - upd_point_XY)
        fp_func_vals[1, index, j] = est_dir.sphere_f(upd_point_XY, minimizer, matrix)
    index += 1

  6%|████▉                                                                             | 6/100 [00:35<09:22,  5.99s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
ticks = max_func_evals_t_list

In [ ]:
import matplotlib.ticker as ticker
def set_box_color(bp, color):
    plt.setp(bp['boxes'], color=color)
    plt.setp(bp['whiskers'], color=color)
    plt.setp(bp['caps'], color=color)
    plt.setp(bp['medians'], color=color)

def create_boxplots(arr1, arr2, arr3, label1, label2, label3, m, n,
                    max_func_evals_t, noise_sd,
                    title):
    plt.figure(figsize=(10, 5))
    if title == 'func_vals':
        plt.yscale("log")
        plt.ylim(0.01, np.max(arr1) + 1000)
    bpl = plt.boxplot(arr1.T,
                      positions=np.array(range(len(arr1)))*3.0-0.7)
    bpc = plt.boxplot(arr2.T,
                      positions=np.array(range(len(arr2)))*3.0+0)
    bpr = plt.boxplot(arr3.T,
                      positions=np.array(range(len(arr3)))*3.0+0.7)
    set_box_color(bpl, 'green')
    set_box_color(bpc, 'purple')
    set_box_color(bpr, 'navy')
    plt.plot([], c='green', label=label1)
    plt.plot([], c='purple', label=label2)
    plt.plot([], c='navy', label=label3)
    plt.legend(bbox_to_anchor=(0.99, 1.025), loc='upper left',
               prop={'size': 12})
    plt.xlabel(r'$u$', size=15)
    plt.xticks(np.arange(0, len(ticks) * 3, 3), ticks, size=15)
    plt.yticks(fontsize=15)
    plt.tight_layout()
#     plt.savefig('%s_m_%s_n_%s_func_evals_%s_%s_%s_sd_%s_diff_lambda_max.pdf' %
#                 (title, m, n, max_func_evals_t, choice_t, no_vars_t, noise_sd))

In [ ]:
labels = [[r'$||x_1 - x_{*}||$',
         r'$||x_{K}^{(LS)} - x_{*}||$',
         r'$||x_{K}^{(MY)} - x_{*}||$'],
        [r'$f(x_1)$',
         r'$f(x_{K}^{(LS)})$',
         r'$f(x_{K}^{(MY)})$']]

In [ ]:
create_boxplots(sp_norms, fp_norms[0], fp_norms[1], labels[0][0], labels[0][1], labels[0][2], m, n,
                max_func_evals_t, noise_sd, 'norms')

In [ ]:
np.median(fp_norms[1][0])

In [ ]:
np.median(fp_norms[1][1])

In [ ]:
np.median(fp_norms[1][2])

In [ ]:
np.median(fp_norms[1][3])

In [ ]:
create_boxplots(sp_func_vals, fp_func_vals[0], fp_func_vals[1], labels[1][0], labels[1][1], labels[1][2], m, n,
                max_func_evals_t, noise_sd, 'func_vals')

In [ ]:
np.median(fp_func_vals[0])

In [ ]:
np.median(fp_func_vals[1][0])

In [ ]:
np.median(fp_func_vals[1][1])

In [ ]:
np.median(fp_func_vals[1][2])

In [ ]:
np.median(fp_func_vals[1][3])

In [38]:
ls_func_vals = np.zeros((len(max_func_evals_t_list)))
xy_func_vals = np.zeros((len(max_func_evals_t_list)))
ls_norms = np.zeros((len(max_func_evals_t_list)))
xy_norms = np.zeros((len(max_func_evals_t_list)))
for j in range(len(max_func_evals_t_list)):
    ls_func_vals[j] = np.median(fp_func_vals[0][j])
    xy_func_vals[j] = np.median(fp_func_vals[1][j])
    ls_norms[j] = np.median(fp_norms[0][j])
    xy_norms[j] = np.median(fp_norms[1][j])

In [39]:
ls_func_vals

array([391.65618919, 385.80610322, 385.80162487, 385.80162487,
       385.80107915])

In [40]:
xy_func_vals

array([166.5185144 , 163.65326229, 158.66902045, 156.64402994,
       156.64402994])

In [41]:
ls_norms

array([27.98771305, 27.77790856, 27.77774733, 27.77774733, 27.77772769])

In [42]:
xy_norms

array([18.24929942, 18.09160667, 17.81397957, 17.6999441 , 17.6999441 ])

In [ ]:
def set_box_color(bp, color):
    plt.setp(bp['boxes'], color=color)
    plt.setp(bp['whiskers'], color=color)
    plt.setp(bp['caps'], color=color)
    plt.setp(bp['medians'], color=color)

def create_boxplots_funv_evals(arr1, arr2, m, n, choice_t, noise_sd, option,
                               ticks, max_func_evals_t):
    plt.figure(figsize=(10, 5))
    bpl = plt.boxplot(arr1.T,
                      positions=np.array(range(len(arr1)))*2.0-0.4)
    bpr = plt.boxplot(arr2.T,
                      positions=np.array(range(len(arr2)))*2.0+0.4)
    set_box_color(bpl, '#D7191C')
    set_box_color(bpr, '#2C7BB6')
    plt.plot([], c='#D7191C', label=r'$\gamma_k$')
    plt.plot([], c='#2C7BB6', label=r'$s_k$')
    plt.legend(bbox_to_anchor=(0.99, 1.025), loc='upper left',
               prop={'size': 15})
    plt.xlabel(r'$u$', size=15)
    plt.xticks(range(0, len(ticks) * 2, 2), ticks, size=15)
    plt.yticks(fontsize=15)
    plt.ylim(0, max_func_evals_t)
    plt.tight_layout()
#     plt.savefig('func_evals_m_%s_n_%s_%s_sd_%s_diff_lambda_max_%s_%s.pdf' %
#                 (m, n, choice_t, noise_sd, option, select_var_t))
    

    

In [ ]:
create_boxplots_funv_evals(func_evals_step[0], func_evals_dir[0], m, n, choice_t, noise_sd, 'LS',
                               ticks, max_func_evals_t)

In [ ]:
create_boxplots_funv_evals(func_evals_step[1], func_evals_dir[1],m, n, choice_t, noise_sd, 'XY',
                               ticks, max_func_evals_t)

In [ ]:
func_evals_step[0][0] + func_evals_dir[0][0]

In [ ]:
(func_evals_step[1][0] + func_evals_dir[1][0]) - (func_evals_step[0][0] + func_evals_dir[0][0])

# NEW

In [ ]:
labels = [[r'$\frac{||x_{K}^{(LS)} - x_{*}||}{||x_1 - x_{*}||}$',
         r'$\frac{||x_{K}^{(MY)} - x_{*}||}{||x_1 - x_{*}||}$'],
        [r'$\frac{f(x_{K}^{(LS)})}{f(x_1)}$',
         r'$\frac{f(x_{K}^{(MY)})}{f(x_1)}$']]

In [ ]:
import matplotlib.ticker as ticker
def set_box_color(bp, color):
    plt.setp(bp['boxes'], color=color)
    plt.setp(bp['whiskers'], color=color)
    plt.setp(bp['caps'], color=color)
    plt.setp(bp['medians'], color=color)

def create_boxplots_ratio(arr1, arr2, label1, label2, m, n,
                            max_func_evals_t, choice_t, no_vars_t, noise_sd,
                            title):
    plt.figure(figsize=(10, 5))
    plt.ylim(0,1)
    bpl = plt.boxplot(arr1.T,
                      positions=np.array(range(len(arr1)))*2.0-0.4)
    bpr = plt.boxplot(arr2.T,
                      positions=np.array(range(len(arr2)))*2.0+0.4)
    set_box_color(bpl, 'green')
    set_box_color(bpr, 'navy')
    plt.plot([], c='green', label=label1)
    plt.plot([], c='navy', label=label2)
    plt.legend(bbox_to_anchor=(0.99, 1.025), loc='upper left',
               prop={'size': 17})
    plt.xlabel(r'$u$', size=15)
    plt.xticks(np.arange(0, len(ticks) * 2, 2), ticks, size=15)
    plt.yticks(fontsize=15)
    plt.tight_layout()
#     plt.savefig('%s_m_%s_n_%s_func_evals_%s_%s_%s_sd_%s_diff_lambda_max.pdf' %
#                 (title, m, n, max_func_evals_t, choice_t, no_vars_t, noise_sd))

In [ ]:
create_boxplots_ratio(fp_norms[0] / sp_norms, fp_norms[1] / sp_norms, labels[0][0], labels[0][1], m, n,
                max_func_evals_t, choice_t, no_vars_t, noise_sd, 'func_vals')

In [ ]:
create_boxplots_ratio(fp_func_vals[0] / sp_func_vals, fp_func_vals[1] / sp_func_vals, labels[1][0], labels[1][1], m, n,
                max_func_evals_t, choice_t, no_vars_t, noise_sd, 'func_vals')

In [ ]:
def create_scatter_plot(arr1, arr2, arr1_title, arr2_title, labels, title, m, n, max_func_evals_t, choice_t,
                        no_vars_t, noise_sd):
    plt.figure(figsize=(10, 5))
    max_num = max(np.max(arr1), np.max(arr2)) + 10
    plt.xlim(0, max_num)
    plt.ylim(0, max_num)
    plt.xlabel(arr1_title, size=15)
    plt.ylabel(arr2_title, size=15)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    color_list = ['red', 'blue', 'purple', 'green', 'orange']
    for j in range(arr1.shape[0]):
        plt.scatter(arr1[j], arr2[j], marker='*', color=color_list[j])
        plt.plot([], c=color_list[j], label=labels[j])
    plt.legend(bbox_to_anchor=(0.99, 1.025), loc='upper left',
               prop={'size': 15})
    plt.tight_layout()
    #     plt.savefig('%s_m_%s_n_%s_func_evals_%s_%s_%s_sd_%s_diff_lambda_max.pdf' %
#                 (title, m, n, max_func_evals_t, choice_t, no_vars_t, noise_sd))

In [ ]:
arr_title = [[r'$||x_{K}^{(LS)} - x_{*}||$',
             r'$||x_{K}^{(MY)} - x_{*}||$'],
            [r'$f(x_{K}^{(LS)})$',
             r'$f(x_{K}^{(MY)})$']]

In [ ]:
labels = []
for j in range(len(max_func_evals_t_list)):
    labels.append(r'$u = %s$' % max_func_evals_t_list[j])
title = 'func_vals_scatter'
create_scatter_plot(fp_func_vals[0], fp_func_vals[1], arr_title[1][0], arr_title[1][1], labels,
                   title, m, n, max_func_evals_t, choice_t, no_vars_t, noise_sd)

In [ ]:
labels = []
for j in range(len(max_func_evals_t_list)):
    labels.append(r'$u = %s$' % max_func_evals_t_list[j])
title = 'func_vals_scatter'
create_scatter_plot(fp_norms[0], fp_norms[1], arr_title[0][0], arr_title[0][1], labels,
                   title, m, n, max_func_evals_t, choice_t, no_vars_t, noise_sd)